In [ ]:
from pan3d.explorers.data_viewer import XArrayViewer
import xarray as xr
from xarray.tutorial import open_dataset

In [ ]:
dataset = xr.open_dataset("../example_dataset.nc", engine="netcdf4")

In [ ]:
dataset = open_dataset("eraint_uvz")

In [ ]:
viewer = XArrayViewer()
await viewer.ui.ready
viewer.xarray = dataset
viewer.computed = {
    "_use_scalars": ["u", "v"], # optional if empty
    "_use_vectors": [], # optional if empty
    "vec": "u*iHat + v*jHat",
    "mag": "u*u + v*v",
}
source = viewer.source

In [ ]:
viewer.ui

In [ ]:
import pyvista as pv

plotter = pv.Plotter()
plotter.add_mesh(source, scalars="mag", cmap='coolwarm')
plotter.show()

In [ ]:
import ipywidgets as widgets

def update_level(change):
    source.slicing = {
        **source.slicing,
        source.z: change.new, 
    }
    viewer.reset_camera()
    plotter.render()

def update_resolution(change):
    v = source.slicing
    v[source.x] = [0, -1, change.new]
    v[source.y] = [0, -1, change.new]
    source.slicing = v
    viewer.reset_camera()
    plotter.render()

slider_level = widgets.IntSlider(value=0, min=0, max=(viewer.xarray[viewer.source.z].size - 1), step=1, description=viewer.source.z)
slider_level.observe(update_level, names='value')
slider_resolution = widgets.IntSlider(value=1, min=1, max=10, step=1, description="resolution")
slider_resolution.observe(update_resolution, names='value')
widgets.HBox([slider_level, slider_resolution])

In [ ]:
print(source)

In [ ]:
from pan3d.xarray.algorithm import vtkXArrayRectilinearSource

algo = vtkXArrayRectilinearSource(dataset)
algo.computed = {
    "_use_scalars": ["u", "v"], # optional if empty
    "_use_vectors": [], # optional if empty
    "vec": "u*iHat + v*jHat",
    "mag": "u*u + v*v",
}